In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import os
import pickle
import gc
from tensorflow.python.keras import layers, Sequential,losses, metrics, optimizers, callbacks, models
from tensorflow.python.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.python.keras.models import Model
from tensorflow.python.keras.applications import vgg16, resnet, densenet, mobilenet
from tensorflow.python.keras.optimizer_v2 import adam

2022-01-04 22:56:09.774661: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0


In [2]:
image_height = 48
image_width = 48
emotions_count = 8
emotion_labels = ['neutral', 'happiness', 'surprise', 'sadness', 'anger', 'disgust', 'fear', 'contempt']

In [3]:
image_path = "./dataset_f+r20/images.npy"
emotion_path = "./dataset_f+r20/emotions_multi.npy"

images = np.load(image_path)
images = tf.convert_to_tensor(images)
images = layers.Rescaling(1./127.5, offset=-1)(images)
images = tf.image.grayscale_to_rgb(images)

emotions = np.load(emotion_path)
emotions = tf.convert_to_tensor(emotions)
    
training_samples = 28317 * 4
validation_samples = 3541 * 4
training_size = training_samples + validation_samples
    
training_images = images[:training_size]
test_images = images[training_size:]
training_emotions = emotions[:training_size]
test_emotions = emotions[training_size:]

2022-01-04 22:56:35.625377: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2022-01-04 22:56:35.636516: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2022-01-04 22:56:36.210770: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:941] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-01-04 22:56:36.211451: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 0 with properties: 
pciBusID: 0000:05:00.0 name: GeForce RTX 2080 Ti computeCapability: 7.5
coreClock: 1.545GHz coreCount: 68 deviceMemorySize: 10.76GiB deviceMemoryBandwidth: 573.69GiB/s
2022-01-04 22:56:36.211500: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
2022-01-04 22:56:36.234734: I tensorflow/stream_executor/platform/defau

In [4]:
tf.config.run_functions_eagerly(True)
def model_acc(y_true, y_pred):
    size = y_true.shape[0]
    acc = 0
    for i in range(size):
        true = y_true[i]
        pred = y_pred[i]           
        index_max = tf.argmax(pred).numpy()
        if true[index_max].numpy()==tf.reduce_max(true).numpy():
            acc += 1
    return acc/size

In [5]:
def train(model, learning_rate, loss, num_epochs, batch_size):
    model.compile(optimizer=adam.Adam(learning_rate=learning_rate), 
                  loss=loss, 
                  metrics = [model_acc])
    history = model.fit(x=training_images,
                        y=training_emotions,
                        batch_size=batch_size,
                        epochs=num_epochs,
                        validation_data=(test_images, test_emotions))
    del model
    gc.collect()
    return history

In [6]:
def create_model(base_model):
    base_model.trainable=True
    return Sequential([
        base_model,
        layers.GlobalAveragePooling2D(),
        layers.Dense(4096, activation='relu'),
        layers.Dense(4096, activation='relu'),
        layers.Dense(emotions_count, activation='softmax'),
    ])

In [10]:
if not os.path.isdir('./results/'):
    os.mkdir('./results/')

num_epochs = 40
batch_size = 32
loss = losses.MeanSquaredError()

learning_rates = [1e-5, 2e-5, 2e-4, 5e-4, 1e-3]
history_save_paths = ['./results/1e-5.txt',
                      './results/2e-5.txt',
                      './results/2e-4.txt',
                      './results/5e-4.txt',
                      './results/1e-3.txt']
for i in range(5):
    base_model = vgg16.VGG16(include_top=False, weights="imagenet", input_shape=(48,48,3))
    model = create_model(base_model)
    history = train(model, learning_rates[i], loss, num_epochs, batch_size)
    with open(history_save_paths[i], 'wb') as file_pi:
        pickle.dump(history.history, file_pi)

Epoch 1/40
3983/3983 [==============================] - 506s 127ms/step - loss: 0.0467 - model_acc: 0.5405 - val_loss: 0.0268 - val_model_acc: 0.7064
Epoch 2/40
3983/3983 [==============================] - 495s 124ms/step - loss: 0.0258 - model_acc: 0.7165 - val_loss: 0.0214 - val_model_acc: 0.7630
Epoch 3/40
3983/3983 [==============================] - 499s 125ms/step - loss: 0.0206 - model_acc: 0.7675 - val_loss: 0.0196 - val_model_acc: 0.7738
Epoch 4/40
3983/3983 [==============================] - 502s 126ms/step - loss: 0.0175 - model_acc: 0.7955 - val_loss: 0.0182 - val_model_acc: 0.7831
Epoch 5/40
3983/3983 [==============================] - 493s 124ms/step - loss: 0.0154 - model_acc: 0.8153 - val_loss: 0.0179 - val_model_acc: 0.7850
Epoch 6/40
3983/3983 [==============================] - 500s 126ms/step - loss: 0.0135 - model_acc: 0.8353 - val_loss: 0.0174 - val_model_acc: 0.8008
Epoch 7/40
3983/3983 [==============================] - 477s 120ms/step - loss: 0.0120 - model_acc: 